<div style='text-align: center;'>
    <img src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQzJzIHdangJTrH2mFXFgsLjuLCjpfXXwbxg&usqp=CAU' width='100'/>
    <h1>Sharif University of Technology</h1>
    <h2>Natural Language Processing</h2>
    <h3>Final Project</h3>
    <h4>Spoiler classification and summary generation</h4>
    <p><strong>Authors:</strong> Abolfazl eshagh</p>
</div>
<hr/>


In [61]:
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Trainer, TrainingArguments
import gc
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

**Load Dataset**

In [3]:
movie_details = pd.read_json('/kaggle/input/imdb-spoiler-dataset/IMDB_movie_details.json', lines=True)
reviews = pd.read_json('/kaggle/input/imdb-spoiler-dataset/IMDB_reviews.json', lines=True)

In [11]:
movie_details

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
...,...,...,...,...,...,...,...
1567,tt0289879,Evan Treborn grows up in a small town with his...,1h 53min,"[Sci-Fi, Thriller]",7.7,2004-01-23,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1568,tt1723811,Brandon is a 30-something man living in New Yo...,1h 41min,[Drama],7.2,2012-01-13,"Brandon (Michael Fassbender) is a successful, ..."
1569,tt5013056,Evacuation of Allied soldiers from the British...,1h 46min,"[Action, Drama, History]",8.1,2017-07-21,The film alternates between three different pe...
1570,tt0104014/,"For a while now, beautiful 24-year-old Diana B...",1h 33min,"[Comedy, Drama]",5.3,1992-02-21,


In [14]:
movie_details.loc[0]

movie_id                                                 tt0105112
plot_summary     Former CIA analyst, Jack Ryan is in England wi...
duration                                                  1h 57min
genre                                           [Action, Thriller]
rating                                                         6.9
release_date                                            1992-06-05
plot_synopsis    Jack Ryan (Ford) is on a "working vacation" in...
Name: 0, dtype: object

In [16]:
movie_details.columns

Index(['movie_id', 'plot_summary', 'duration', 'genre', 'rating',
       'release_date', 'plot_synopsis'],
      dtype='object')

In [17]:
movie_details.loc[0]['plot_synopsis']

'Jack Ryan (Ford) is on a "working vacation" in London with his family. He has retired from the CIA and is a Professor at the US Naval Academy. He is seen delivering a lecture at the Royal Naval Academy in London.Meanwhile, Ryan\'s wife Cathy and daughter Sally are sightseeing near Buckingham Palace. Sally and Cathy come upon a British Royal Guard, and Sally tries to get the guard to react by doing an improvised tap dance in front of him. She\'s impressed when the guard, trained to ignore distraction, doesn\'t react at all, and they leave.As Sally and Cathy walk away from the guard, en route to rendezvous with Ryan, they walk by a stolen cab, in which sit three Ulster Liberation Army terrorists: Kevin O\'Donnell, the driver, as well as Sean Miller (Sean Bean) and his younger brother Patrick. The three are loading bullets into their guns as they prepare to carry out a scheduled ambush on Lord William Holmes, British Secretary of State for Northern Ireland and a distant member of the Bri

In [18]:
movie_details.loc[0]['plot_summary']

"Former CIA analyst, Jack Ryan is in England with his family on vacation when he suddenly witnesses an explosion outside Buckingham Palace. It is revealed that some people are trying to abduct a member of the Royal Family but Jack intervenes, killing one of them and capturing the other, and stops the plan in its tracks. Afterwards, he learns that they're Irish revolutionaries and the two men are brothers. During his court hearing the one that's still alive vows to get back at Jack but is sentenced and that seems to be the end of it. However, whilst the man is being transported, he is broken out. Jack learns of this but doesn't think there's anything to worry about. But, when he is at the Naval Academy someone tries to kill him. He learns that they are also going after his family and so he rushes to find them, safe but having also been the victims of a failed assassination. That's when Jack decides to rejoin the CIA, and they try to find the man before he makes another attempt.         

In [19]:
reviews

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
573908,8 August 1999,tt0139239,ur0100166,False,"Go is wise, fast and pure entertainment. Assem...",10,The best teen movie of the nineties
573909,31 July 1999,tt0139239,ur0021767,False,"Well, what shall I say. this one´s fun at any ...",9,Go - see the movie
573910,20 July 1999,tt0139239,ur0392750,False,"Go is the best movie I have ever seen, and I'v...",10,It's the best movie I've ever seen
573911,11 June 1999,tt0139239,ur0349105,False,Call this 1999 teenage version of Pulp Fiction...,3,Haven't we seen this before?


In [22]:
sample_review = reviews.loc[0]
sample_review['review_text']

'In its Oscar year, Shawshank Redemption (written and directed by Frank Darabont, after the novella Rita Hayworth and the Shawshank Redemption, by Stephen King) was nominated for seven Academy Awards, and walked away with zero. Best Picture went to Forrest Gump, while Shawshank and Pulp Fiction were "just happy to be nominated." Of course hindsight is 20/20, but while history looks back on Gump as a good film, Pulp and Redemption are remembered as some of the all-time best. Pulp, however, was a success from the word "go," making a huge splash at Cannes and making its writer-director an American master after only two films. For Andy Dufresne and Co., success didn\'t come easy. Fortunately, failure wasn\'t a life sentence.After opening on 33 screens with take of $727,327, the $25M film fell fast from theatres and finished with a mere $28.3M. The reasons for failure are many. Firstly, the title is a clunker. While iconic to fans today, in 1994, people knew not and cared not what a \'Shaws

In [23]:
sample_review['review_summary']

'A classic piece of unforgettable film-making.'

In [27]:
movie_details = pd.DataFrame(movie_details)
movie_details

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
...,...,...,...,...,...,...,...
1567,tt0289879,Evan Treborn grows up in a small town with his...,1h 53min,"[Sci-Fi, Thriller]",7.7,2004-01-23,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1568,tt1723811,Brandon is a 30-something man living in New Yo...,1h 41min,[Drama],7.2,2012-01-13,"Brandon (Michael Fassbender) is a successful, ..."
1569,tt5013056,Evacuation of Allied soldiers from the British...,1h 46min,"[Action, Drama, History]",8.1,2017-07-21,The film alternates between three different pe...
1570,tt0104014/,"For a while now, beautiful 24-year-old Diana B...",1h 33min,"[Comedy, Drama]",5.3,1992-02-21,


In [28]:
reviews = pd.DataFrame(reviews)
reviews

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
573908,8 August 1999,tt0139239,ur0100166,False,"Go is wise, fast and pure entertainment. Assem...",10,The best teen movie of the nineties
573909,31 July 1999,tt0139239,ur0021767,False,"Well, what shall I say. this one´s fun at any ...",9,Go - see the movie
573910,20 July 1999,tt0139239,ur0392750,False,"Go is the best movie I have ever seen, and I'v...",10,It's the best movie I've ever seen
573911,11 June 1999,tt0139239,ur0349105,False,Call this 1999 teenage version of Pulp Fiction...,3,Haven't we seen this before?


In [37]:
movie_details = movie_details[movie_details['plot_synopsis'] != ""]
movie_details

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
5,tt0286716,"Bruce Banner, a brilliant scientist with a clo...",2h 18min,"[Action, Sci-Fi]",5.7,2003-06-20,Bruce Banner (Eric Bana) is a research scienti...
...,...,...,...,...,...,...,...
1563,tt0120655,An abortion clinic worker with a special herit...,2h 10min,"[Adventure, Comedy, Drama]",7.3,1999-11-12,The film opens with a homeless man (Bud Cort) ...
1565,tt0276751,Twelve year old Marcus Brewer lives with his c...,1h 41min,"[Comedy, Drama, Romance]",7.1,2002-05-17,Will Freeman (Hugh Grant) is a 38-year-old bac...
1567,tt0289879,Evan Treborn grows up in a small town with his...,1h 53min,"[Sci-Fi, Thriller]",7.7,2004-01-23,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1568,tt1723811,Brandon is a 30-something man living in New Yo...,1h 41min,[Drama],7.2,2012-01-13,"Brandon (Michael Fassbender) is a successful, ..."


In [39]:
movie_details = pd.DataFrame(movie_details)

In [40]:
movie_details

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
5,tt0286716,"Bruce Banner, a brilliant scientist with a clo...",2h 18min,"[Action, Sci-Fi]",5.7,2003-06-20,Bruce Banner (Eric Bana) is a research scienti...
...,...,...,...,...,...,...,...
1563,tt0120655,An abortion clinic worker with a special herit...,2h 10min,"[Adventure, Comedy, Drama]",7.3,1999-11-12,The film opens with a homeless man (Bud Cort) ...
1565,tt0276751,Twelve year old Marcus Brewer lives with his c...,1h 41min,"[Comedy, Drama, Romance]",7.1,2002-05-17,Will Freeman (Hugh Grant) is a 38-year-old bac...
1567,tt0289879,Evan Treborn grows up in a small town with his...,1h 53min,"[Sci-Fi, Thriller]",7.7,2004-01-23,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1568,tt1723811,Brandon is a 30-something man living in New Yo...,1h 41min,[Drama],7.2,2012-01-13,"Brandon (Michael Fassbender) is a successful, ..."


In [41]:
sample1 = movie_details.loc[1]
sample1

movie_id                                                 tt1204975
plot_summary     Billy (Michael Douglas), Paddy (Robert De Niro...
duration                                                  1h 45min
genre                                                     [Comedy]
rating                                                         6.6
release_date                                            2013-11-01
plot_synopsis    Four boys around the age of 10 are friends in ...
Name: 1, dtype: object

In [42]:
sample1['plot_summary']

"Billy (Michael Douglas), Paddy (Robert De Niro), Archie (Morgan Freeman) and Sam (Kevin Kline) have been best friends since childhood. So when Billy, the group's sworn bachelor, finally proposes to his thirty-something (of course) girlfriend, the four head to Las Vegas with a plan to stop acting their age and relive their glory days. However, upon arriving, the four quickly realize that the decades have transformed Sin City and tested their friendship in ways they never imagined. The Rat Pack may have once played the Sands and Cirque du Soleil may now rule the Strip, but it's these four who are taking over Vegas.                Written by\nProduction"

In [45]:
sample1['plot_synopsis']

'Four boys around the age of 10 are friends in Brooklyn during the \'50s. They have a nickname, the Flatbush Four, and also pal around with a girl. One day they get into a scuffle at a store with a young thug, and steal a bottle of scotch as they run away.58 years later: Sam has retired with his wife to Naples, FL; Archie is living with his son in New Jersey; Paddy lives alone in Brooklyn after his wife Sophie died a year earlier; Billy is wealthy and living in Malibu, CA with a young woman named Lisa.Billy goes to the funeral for his friend Ronnie, and in the process of eulogizing him, while commenting on the shortness of life and love, he spontaneously proposes to Lisa.Billy conference calls Sam and Archie to tell them he is getting married that weekend in Las Vegas, and they not only want to come to the wedding, they want to throw him a bachelor party the night before. All three of them dread calling Paddy though.Sam\'s wife sends him off to Vegas with a condom, telling him he can d

In [47]:
reviews = reviews[reviews['review_summary'] != '']
reviews

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
573908,8 August 1999,tt0139239,ur0100166,False,"Go is wise, fast and pure entertainment. Assem...",10,The best teen movie of the nineties
573909,31 July 1999,tt0139239,ur0021767,False,"Well, what shall I say. this one´s fun at any ...",9,Go - see the movie
573910,20 July 1999,tt0139239,ur0392750,False,"Go is the best movie I have ever seen, and I'v...",10,It's the best movie I've ever seen
573911,11 June 1999,tt0139239,ur0349105,False,Call this 1999 teenage version of Pulp Fiction...,3,Haven't we seen this before?


In [48]:
review1 = reviews.loc[1]
review1

review_date                                        6 September 2000
movie_id                                                  tt0111161
user_id                                                   ur0842118
is_spoiler                                                     True
review_text       The Shawshank Redemption is without a doubt on...
rating                                                           10
review_summary           Simply amazing. The best film of the 90's.
Name: 1, dtype: object

In [50]:
review1['review_text']

'The Shawshank Redemption is without a doubt one of the most brilliant movies I have ever seen. Similar to The Green Mile in many respects (and better than it in almost all of them), these two movies have shown us that Stephen King is a master not only of horror but also of prose that shakes the soul and moves the heart. The plot is average, but King did great things with it in his novella that are only furthered by the direction, and the acting is so top-rate it\'s almost scary.Tim Robbins plays Andy Dufrane, wrongly imprisoned for 20 years for the murder of his wife. The story focuses on Andy\'s relationship with "Red" Redding (Morgan Freeman, in probably his best role) and his attempts to escape from Shawshank. Bob Gunton is positively evil and frightening as Warden Norton, and there are great performances and cameos all around; the most prominent one being Gil Bellows (late as Billy of Ally McBeal) as Tommy, a fellow inmate of Andy\'s who suffers under the iron will of Norton.If yo

In [51]:
review1['review_summary']

"Simply amazing. The best film of the 90's."

In [52]:
movie_details.columns, reviews.columns

(Index(['movie_id', 'plot_summary', 'duration', 'genre', 'rating',
        'release_date', 'plot_synopsis'],
       dtype='object'),
 Index(['review_date', 'movie_id', 'user_id', 'is_spoiler', 'review_text',
        'rating', 'review_summary'],
       dtype='object'))

In [53]:
movie_details = movie_details.drop(columns = ['movie_id', 'duration', 'genre', 'rating', 'release_date'])
reviews = reviews.drop(columns = ['review_date', 'movie_id', 'user_id', 'is_spoiler', 'rating'])
movie_details

,plot_summary,plot_synopsis
0,"Former CIA analyst, Jack Ryan is in England wi...","Jack Ryan (Ford) is on a ""working vacation"" in..."
1,"Billy (Michael Douglas), Paddy (Robert De Niro...",Four boys around the age of 10 are friends in ...
3,"Fred C. Dobbs and Bob Curtin, both down on the...",Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,Tracy Flick is running unopposed for this year...,Jim McAllister (Matthew Broderick) is a much-a...
5,"Bruce Banner, a brilliant scientist with a clo...",Bruce Banner (Eric Bana) is a research scienti...
...,...,...
1563,An abortion clinic worker with a special herit...,The film opens with a homeless man (Bud Cort) ...
1565,Twelve year old Marcus Brewer lives with his c...,Will Freeman (Hugh Grant) is a 38-year-old bac...
1567,Evan Treborn grows up in a small town with his...,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1568,Brandon is a 30-something man living in New Yo...,"Brandon (Michael Fassbender) is a successful, ..."


In [54]:
reviews

,review_text,review_summary
0,"In its Oscar year, Shawshank Redemption (writt...",A classic piece of unforgettable film-making.
1,The Shawshank Redemption is without a doubt on...,Simply amazing. The best film of the 90's.
2,I believe that this film is the best story eve...,The best story ever told on film
3,"**Yes, there are SPOILERS here**This film has ...",Busy dying or busy living?
4,At the heart of this extraordinary movie is a ...,"Great story, wondrously told and acted"
...,...,...
573908,"Go is wise, fast and pure entertainment. Assem...",The best teen movie of the nineties
573909,"Well, what shall I say. this one´s fun at any ...",Go - see the movie
573910,"Go is the best movie I have ever seen, and I'v...",It's the best movie I've ever seen
573911,Call this 1999 teenage version of Pulp Fiction...,Haven't we seen this before?


In [57]:
from sklearn.model_selection import train_test_split
plot_dataset = movie_details[['plot_summary', 'plot_synopsis']]

X = plot_dataset['plot_summary']
y = plot_dataset['plot_synopsis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_set = pd.DataFrame({'plot_summary': X_train, 'plot_synopsis': y_train})
test_set = pd.DataFrame({'plot_summary': X_test, 'plot_synopsis': y_test})

In [58]:
train_set

,plot_summary,plot_synopsis
1358,Eccentric actor Daniel Hillard is an amusing a...,Daniel Hillard is a voice actor living in San ...
944,Volcanologist Harry Dalton and mayor Rachel Wa...,"Dr. Harry Dalton, a volcanologist with the Uni..."
1510,Straight from the pages of a pulp comic from a...,The movie takes place in Los Angeles in the ye...
696,Scarlett is a woman who can deal with a nation...,"The film opens in Tara, a cotton plantation ow..."
1399,Once again we're plunged into the world of swo...,The wedding for Will Turner and Elizabeth Swan...
...,...,...
1290,Joe Kingman is a huge football star. He's got ...,"Top Boston Rebels football star, Quarterback J..."
1331,"In a brand new Jumanji adventure, four high sc...","In 1996, teenager Alex Vreeke (Mason Guccione)..."
1520,"Over the holidays, Ned, an overprotective but ...",The film starts with a Facetime chat between 3...
1010,Peter Parker has finally managed to piece toge...,"Months after the events of Spider-Man 2, Peter..."


In [59]:
test_set

,plot_summary,plot_synopsis
471,A young boy stumbles into a mysterious girl wh...,&#12302;&#22825;&#31354;&#12398;&#22478;&#1252...
1037,"More than 10 years after 'Terminator 2', John ...","Following the events of ""Terminator 2: Judgmen..."
428,"Ethan (Joseph Gordon-Levitt), Isaac (Seth Roge...","Christmas Eve, 2001Ethan Miller (Joseph Gordon..."
437,Due to his knowledge of the native Bedouin tri...,"In 1935, T. E. Lawrence (Peter O'Toole) is kil..."
311,"In a nursing home, resident Duke reads a roman...","In a modern-day nursing home, an elderly man n..."
...,...,...
125,The final Twilight Saga begins with Bella now ...,Bella awakens to a new world. Her senses are s...
957,Monco is a bounty killer chasing El Indio and ...,The film begins with Colonel Douglas Mortimer ...
637,"In 1893, Gandhi is thrown off a South African ...",The story begins with the assassination of Moh...
1508,After it looks as if she's left his life for g...,(500) Days of Summer is presented in a non-chr...


In [62]:
# Load BART tokenizer and model
model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [63]:
# Tokenize data
train_encodings = tokenizer(train_set['plot_summary'].tolist(), truncation=True, padding="max_length", max_length=256, return_tensors="pt")
train_labels = tokenizer(train_set['plot_synopsis'].tolist(), truncation=True, padding="max_length", max_length=64, return_tensors="pt")
val_encodings = tokenizer(test_set['plot_summary'].tolist(), truncation=True, padding="max_length", max_length=256, return_tensors="pt")
val_labels = tokenizer(test_set['plot_synopsis'].tolist(), truncation=True, padding="max_length", max_length=64, return_tensors="pt")

In [66]:
# Create dataset class
class PlotDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels['input_ids'][idx].clone().detach()
        return item

In [67]:
train_dataset_prepared = PlotDataset(train_encodings, train_labels)
val_dataset = PlotDataset(val_encodings, val_labels)

In [70]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [71]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_prepared,
    eval_dataset=val_dataset,
)

In [72]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,3.397900,3.225342


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.emb

TrainOutput(global_step=536, training_loss=3.587674368673296, metrics={'train_runtime': 180.2582, 'train_samples_per_second': 5.941, 'train_steps_per_second': 2.974, 'total_flos': 580242257215488.0, 'train_loss': 3.587674368673296, 'epoch': 1.0})

In [73]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [74]:
# Save model and tokenizer
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')
torch.save(model.state_dict, 'bart_plot_summary1.pth')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [75]:
# Evaluate model
# model = MBartForConditionalGeneration.from_pretrained('./saved_model')
# tokenizer = MBart50TokenizerFast.from_pretrained('./saved_model')
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# val_dataset = PlotDataset(test_encodings, test_labels)
test_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

all_predictions = []
all_references = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model.generate(input_ids, max_length=64, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
        all_predictions.extend(outputs)
        all_references.extend(labels)

In [76]:
# Decode predictions and references
decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in all_predictions]
decoded_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in all_references]

In [78]:
!pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b93f78fcdad1cc811a0686910d93c6b5304375d0393f3581d55444072e1f2dc3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [81]:
!pip install bert_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00


In [82]:
# Compute metrics
from datasets import load_metric
# rouge_types=["rouge1", "rouge2", "rougeL"]
rouge = load_metric("rouge")
# rouge = rouge(rouge_types)
bleu = load_metric("bleu")
from bert_score import score

rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
bleu_result = bleu.compute(predictions=[pred.split() for pred in decoded_preds], references=[[ref.split()] for ref in decoded_labels])
P, R, F1 = score(decoded_preds, decoded_labels, lang='multilingual')
bertscore_result = {"precision": P.mean().item(), "recall": R.mean().item(), "f1": F1.mean().item()}

results = {"bleu": bleu_result, "rouge": rouge_result, "bertscore": bertscore_result}

# Print results
from prettytable import PrettyTable

def print_results(results):
    table = PrettyTable()
    table.field_names = ["Metric", "Value"]

    bleu_scores = results['bleu']
    table.add_row(["BLEU Score", f"{bleu_scores['bleu']:.4f}"])
    table.add_row(["Brevity Penalty", f"{bleu_scores['brevity_penalty']:.4f}"])
    table.add_row(["Length Ratio", f"{bleu_scores['length_ratio']:.4f}"])
    table.add_row(["Translation Length", bleu_scores['translation_length']])
    table.add_row(["Reference Length", bleu_scores['reference_length']])
    for i, precision in enumerate(bleu_scores['precisions'], 1):
        table.add_row([f"Precisions {i}-gram", f"{precision:.4f}"])

    rouge_scores = results['rouge']
    for key in rouge_scores:
        score = rouge_scores[key].mid
        table.add_row([f"ROUGE-{key} (Precision)", f"{score.precision:.4f}"])
        table.add_row([f"ROUGE-{key} (Recall)", f"{score.recall:.4f}"])
        table.add_row([f"ROUGE-{key} (F-Measure)", f"{score.fmeasure:.4f}"])

    bert_scores = results['bertscore']
    table.add_row(["BERTScore Precision", f"{bert_scores['precision']:.4f}"])
    table.add_row(["BERTScore Recall", f"{bert_scores['recall']:.4f}"])
    table.add_row(["BERTScore F1", f"{bert_scores['f1']:.4f}"])

    print(table)

print_results(results)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

+-----------------------------+--------+
|            Metric           | Value  |
+-----------------------------+--------+
|          BLEU Score         | 0.0229 |
|       Brevity Penalty       | 0.8956 |
|         Length Ratio        | 0.9007 |
|      Translation Length     | 11232  |
|       Reference Length      | 12470  |
|      Precisions 1-gram      | 0.2303 |
|      Precisions 2-gram      | 0.0412 |
|      Precisions 3-gram      | 0.0108 |
|      Precisions 4-gram      | 0.0041 |
|   ROUGE-rouge1 (Precision)  | 0.2832 |
|    ROUGE-rouge1 (Recall)    | 0.2402 |
|   ROUGE-rouge1 (F-Measure)  | 0.2534 |
|   ROUGE-rouge2 (Precision)  | 0.0643 |
|    ROUGE-rouge2 (Recall)    | 0.0522 |
|   ROUGE-rouge2 (F-Measure)  | 0.0556 |
|   ROUGE-rougeL (Precision)  | 0.1917 |
|    ROUGE-rougeL (Recall)    | 0.1617 |
|   ROUGE-rougeL (F-Measure)  | 0.1706 |
| ROUGE-rougeLsum (Precision) | 0.1921 |
|   ROUGE-rougeLsum (Recall)  | 0.1625 |
| ROUGE-rougeLsum (F-Measure) | 0.1714 |
|     BERTScore 

In [83]:
test_set

,plot_summary,plot_synopsis
471,A young boy stumbles into a mysterious girl wh...,&#12302;&#22825;&#31354;&#12398;&#22478;&#1252...
1037,"More than 10 years after 'Terminator 2', John ...","Following the events of ""Terminator 2: Judgmen..."
428,"Ethan (Joseph Gordon-Levitt), Isaac (Seth Roge...","Christmas Eve, 2001Ethan Miller (Joseph Gordon..."
437,Due to his knowledge of the native Bedouin tri...,"In 1935, T. E. Lawrence (Peter O'Toole) is kil..."
311,"In a nursing home, resident Duke reads a roman...","In a modern-day nursing home, an elderly man n..."
...,...,...
125,The final Twilight Saga begins with Bella now ...,Bella awakens to a new world. Her senses are s...
957,Monco is a bounty killer chasing El Indio and ...,The film begins with Colonel Douglas Mortimer ...
637,"In 1893, Gandhi is thrown off a South African ...",The story begins with the assassination of Moh...
1508,After it looks as if she's left his life for g...,(500) Days of Summer is presented in a non-chr...


In [86]:
input_plot = test_set.loc[835]
input_plot['plot_summary']

"Lara Brennan is arrested for murdering her boss with whom she had an argument. It seems she was seen leaving the scene of the crime and her fingerprints were on the murder weapon. Her husband, John would spend the next few years trying to get her released, but there's no evidence that negates the evidence against her. And when the strain of being separated from her family, especially her son, gets to her, John decides to break her out. So he does a lot of research to find a way.                Written by\nrcs0411@yahoo.com"

In [87]:
input_plot['plot_synopsis']

'The credits roll with John Brennan (Russel Crowe) driving a Prius. He has a passenger who is making moaning sounds about how he can\'t breathe. Suddenly, the guy stops moaning.Titles say "Last three years." John and Lara Brennan (Elizabeth Banks), and his brother Mick (Michael Buie) and wife Erit (Moran Atias) are out to dinner and it is not going well. Erit and Lara are fighting over whether it\'s a good idea for women to work for other women and it turns into a real nasty verbal cat fight. Lara has just had a big fight with her boss. John and Lara head home, relieve the sitter, and make out. Next morning, she injects herself with insulin. As she\'s getting ready, she notices a spot on her rain jacket and is washing it out in the sink - just as she realizes that it\'s blood, the police storm in and arrest her for the murder of her boss. Their 3 year old son Luke (Tyler & Toby Green) is bawling.Three years later, John is a community college teacher who is teaching English. He tries to

In [89]:

# Example article for prediction
example_article = input_plot['plot_summary']
input_encoding = tokenizer(example_article, return_tensors="pt", max_length=256, padding="max_length", truncation=True)

print("input plot summary:")
print(example_article)

# Generate title
model.eval()
with torch.no_grad():
    input_encoding = {k: v.to(device) for k, v in input_encoding.items()}
    outputs = model.generate(input_encoding['input_ids'], max_length=64, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    predicted_title = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Predicted plot synsopis:")
print(predicted_title)

input plot summary:
Lara Brennan is arrested for murdering her boss with whom she had an argument. It seems she was seen leaving the scene of the crime and her fingerprints were on the murder weapon. Her husband, John would spend the next few years trying to get her released, but there's no evidence that negates the evidence against her. And when the strain of being separated from her family, especially her son, gets to her, John decides to break her out. So he does a lot of research to find a way.                Written by
rcs0411@yahoo.com
Predicted plot synsopis:
Lara Brennan is arrested for murdering her boss with whom she had an argument. It seems she was seen leaving the scene of the crime and her fingerprints were on the murder weapon. Her husband, John would spend the next few years trying to get her released, but there's no evidence that negates the


**Plot summary generation**

In [90]:
# Tokenize data
train_encodings = tokenizer(train_set['plot_synopsis'].tolist(), truncation=True, padding="max_length", max_length=256, return_tensors="pt")
train_labels = tokenizer(train_set['plot_summary'].tolist(), truncation=True, padding="max_length", max_length=64, return_tensors="pt")
val_encodings = tokenizer(test_set['plot_synopsis'].tolist(), truncation=True, padding="max_length", max_length=256, return_tensors="pt")
val_labels = tokenizer(test_set['plot_summary'].tolist(), truncation=True, padding="max_length", max_length=64, return_tensors="pt")

In [91]:
train_dataset_prepared = PlotDataset(train_encodings, train_labels)
val_dataset = PlotDataset(val_encodings, val_labels)

In [92]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [93]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_prepared,
    eval_dataset=val_dataset,
)

In [94]:
trainer.train()

Step,Training Loss,Validation Loss
500,3.280500,3.175063


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.emb

TrainOutput(global_step=536, training_loss=3.339665957351229, metrics={'train_runtime': 167.2334, 'train_samples_per_second': 6.404, 'train_steps_per_second': 3.205, 'total_flos': 580242257215488.0, 'train_loss': 3.339665957351229, 'epoch': 1.0})

In [95]:
gc.collect()
torch.cuda.empty_cache()

In [96]:
# Save model and tokenizer
model.save_pretrained('./saved_model_plotSummary')
tokenizer.save_pretrained('./saved_model_plotSummary')
torch.save(model.state_dict, 'bart_generating_plot_summary1.pth')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [97]:
# Evaluate model
# model = MBartForConditionalGeneration.from_pretrained('./saved_model')
# tokenizer = MBart50TokenizerFast.from_pretrained('./saved_model')
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# val_dataset = PlotDataset(test_encodings, test_labels)
test_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

all_predictions = []
all_references = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model.generate(input_ids, max_length=64, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
        all_predictions.extend(outputs)
        all_references.extend(labels)

In [98]:
# Decode predictions and references
decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in all_predictions]
decoded_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in all_references]

In [99]:
# Compute metrics
from datasets import load_metric
# rouge_types=["rouge1", "rouge2", "rougeL"]
rouge = load_metric("rouge")
# rouge = rouge(rouge_types)
bleu = load_metric("bleu")
from bert_score import score

rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
bleu_result = bleu.compute(predictions=[pred.split() for pred in decoded_preds], references=[[ref.split()] for ref in decoded_labels])
P, R, F1 = score(decoded_preds, decoded_labels, lang='multilingual')
bertscore_result = {"precision": P.mean().item(), "recall": R.mean().item(), "f1": F1.mean().item()}

results = {"bleu": bleu_result, "rouge": rouge_result, "bertscore": bertscore_result}

# Print results
from prettytable import PrettyTable

def print_results(results):
    table = PrettyTable()
    table.field_names = ["Metric", "Value"]

    bleu_scores = results['bleu']
    table.add_row(["BLEU Score", f"{bleu_scores['bleu']:.4f}"])
    table.add_row(["Brevity Penalty", f"{bleu_scores['brevity_penalty']:.4f}"])
    table.add_row(["Length Ratio", f"{bleu_scores['length_ratio']:.4f}"])
    table.add_row(["Translation Length", bleu_scores['translation_length']])
    table.add_row(["Reference Length", bleu_scores['reference_length']])
    for i, precision in enumerate(bleu_scores['precisions'], 1):
        table.add_row([f"Precisions {i}-gram", f"{precision:.4f}"])

    rouge_scores = results['rouge']
    for key in rouge_scores:
        score = rouge_scores[key].mid
        table.add_row([f"ROUGE-{key} (Precision)", f"{score.precision:.4f}"])
        table.add_row([f"ROUGE-{key} (Recall)", f"{score.recall:.4f}"])
        table.add_row([f"ROUGE-{key} (F-Measure)", f"{score.fmeasure:.4f}"])

    bert_scores = results['bertscore']
    table.add_row(["BERTScore Precision", f"{bert_scores['precision']:.4f}"])
    table.add_row(["BERTScore Recall", f"{bert_scores['recall']:.4f}"])
    table.add_row(["BERTScore F1", f"{bert_scores['f1']:.4f}"])

    print(table)

print_results(results)


+-----------------------------+--------+
|            Metric           | Value  |
+-----------------------------+--------+
|          BLEU Score         | 0.0240 |
|       Brevity Penalty       | 0.9521 |
|         Length Ratio        | 0.9532 |
|      Translation Length     | 12386  |
|       Reference Length      | 12994  |
|      Precisions 1-gram      | 0.2369 |
|      Precisions 2-gram      | 0.0383 |
|      Precisions 3-gram      | 0.0104 |
|      Precisions 4-gram      | 0.0043 |
|   ROUGE-rouge1 (Precision)  | 0.2841 |
|    ROUGE-rouge1 (Recall)    | 0.2697 |
|   ROUGE-rouge1 (F-Measure)  | 0.2754 |
|   ROUGE-rouge2 (Precision)  | 0.0591 |
|    ROUGE-rouge2 (Recall)    | 0.0556 |
|   ROUGE-rouge2 (F-Measure)  | 0.0572 |
|   ROUGE-rougeL (Precision)  | 0.1817 |
|    ROUGE-rougeL (Recall)    | 0.1722 |
|   ROUGE-rougeL (F-Measure)  | 0.1759 |
| ROUGE-rougeLsum (Precision) | 0.1818 |
|   ROUGE-rougeLsum (Recall)  | 0.1721 |
| ROUGE-rougeLsum (F-Measure) | 0.1761 |
|     BERTScore 

In [100]:
input_plot = test_set.loc[835]
input_plot

plot_summary     Lara Brennan is arrested for murdering her bos...
plot_synopsis    The credits roll with John Brennan (Russel Cro...
Name: 835, dtype: object

In [103]:
input_plot['plot_summary']

"Lara Brennan is arrested for murdering her boss with whom she had an argument. It seems she was seen leaving the scene of the crime and her fingerprints were on the murder weapon. Her husband, John would spend the next few years trying to get her released, but there's no evidence that negates the evidence against her. And when the strain of being separated from her family, especially her son, gets to her, John decides to break her out. So he does a lot of research to find a way.                Written by\nrcs0411@yahoo.com"

In [101]:

# Example article for prediction

example_article = input_plot['plot_synopsis']
input_encoding = tokenizer(example_article, return_tensors="pt", max_length=256, padding="max_length", truncation=True)

print("input plot:")
print(example_article)

# Generate title
model.eval()
with torch.no_grad():
    input_encoding = {k: v.to(device) for k, v in input_encoding.items()}
    outputs = model.generate(input_encoding['input_ids'], max_length=64, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    predicted_title = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Predicted plot summary:")
print(predicted_title)

input plot:
The credits roll with John Brennan (Russel Crowe) driving a Prius. He has a passenger who is making moaning sounds about how he can't breathe. Suddenly, the guy stops moaning.Titles say "Last three years." John and Lara Brennan (Elizabeth Banks), and his brother Mick (Michael Buie) and wife Erit (Moran Atias) are out to dinner and it is not going well. Erit and Lara are fighting over whether it's a good idea for women to work for other women and it turns into a real nasty verbal cat fight. Lara has just had a big fight with her boss. John and Lara head home, relieve the sitter, and make out. Next morning, she injects herself with insulin. As she's getting ready, she notices a spot on her rain jacket and is washing it out in the sink - just as she realizes that it's blood, the police storm in and arrest her for the murder of her boss. Their 3 year old son Luke (Tyler & Toby Green) is bawling.Three years later, John is a community college teacher who is teaching English. He t